In [1]:
import gym
import os
import time
import numpy as np
import tensorflow as tf

C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
class PPO_Agent:
    def __init__(self, 
                 sess, 
                 input_shape, 
                 action_num,
                 gamma = 0.99,
                 batch_size = 64,
                 epsilon = 0.2,
                 lamda = 0.95,
                 lr=1e-5):
        self.sess = sess
        self.input_shape = list(input_shape)
        self.action_num = action_num
        self.gamma = gamma
        self.batch_size = batch_size
        self.epsilon = epsilon # clip threshold
        self.lamda = lamda # gae parameter

        self.value_estimator = ValueEstimator(sess, input_shape)
        self._reset(init=True)
        
        with tf.variable_scope("Actor"):
            self.ep_states = tf.placeholder(tf.float32, [None] + self.input_shape)
            self.ep_actions = tf.placeholder(tf.int32, [None]) # actions taken in an episode
            self.ep_advantage = tf.placeholder(tf.float32, [None]) # advantage for each (s,a) pair in an episode

            self.action_logits = self._build_network("policy", self.ep_states)
            self.action_logits_old = self._build_network("policy_old", self.ep_states)
            self.action_prob = tf.nn.softmax(self.action_logits) * tf.one_hot(self.ep_actions, action_num)
            self.action_prob = tf.reduce_sum(self.action_prob, axis=1)
            self.action_prob_old = tf.nn.softmax(self.action_logits_old) * tf.one_hot(self.ep_actions, action_num)
            self.action_prob_old = tf.reduce_sum(self.action_prob_old, axis=1)
            
            self.act_prob = tf.nn.softmax(self.action_logits_old) # used for collecting trajactories
            
            self.policy_param = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="Actor/policy")
            self.old_param = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="Actor/policy_old")
            self.assign_ops = [tf.assign(dest_var, src_var) for dest_var, src_var in zip(self.old_param, self.policy_param)]
            
            self.ratio = tf.divide(self.action_prob, self.action_prob_old)
            self.ratio_clip = tf.clip_by_value(self.ratio, 1-epsilon, 1+epsilon)
            self.clip_loss = tf.minimum(self.ratio*self.ep_advantage, self.ratio_clip*self.ep_advantage)
        
            optimizer = tf.train.AdamOptimizer(learning_rate=lr)
            self.train_step = optimizer.minimize(-self.clip_loss, var_list=self.policy_param)
        
    def _build_network(self, scope, X):
        with tf.variable_scope(scope):
            fc1 = tf.layers.dense(X, 128, activation=tf.nn.relu)
            fc2 = tf.layers.dense(fc1, 256, activation=tf.nn.relu)
            fc3 = tf.layers.dense(fc2, 256, activation=tf.nn.relu)
            action_logits = tf.layers.dense(fc3, self.action_num)
        return action_logits
    
    def _reset(self, init=False):
        self.reward_list = []
        self.state_list = []
        self.action_list = []
        if not init:
            self.sess.run(self.assign_ops)
        
    def act(self, state, training=True):
        state = np.reshape(state, [1] + self.input_shape)
        probs = sess.run(self.act_prob, feed_dict={self.ep_states:state})[0]
        if training:
            action = np.random.choice(self.action_num, p=probs)
        else:
            action = np.argmax(probs)
        return action
    
    def observe(self, state, action, reward):
        self.reward_list.append(reward)
        self.state_list.append(state)
        self.action_list.append(action)
    
    def train(self):
        discounted_reward = self._caculate_reward()
        Vs = self.value_estimator.predict(self.state_list)
        #Vs_ = np.append(Vs[1:], 0)
        advantages = discounted_reward - Vs
        #advantages = self.gae(Vs, Vs_)
        
        # train actor
        for _ in range(4):
            sess.run(self.train_step, feed_dict={self.ep_states:self.state_list,
                                                 self.ep_actions:self.action_list,
                                                 self.ep_advantage:advantages})

        for _ in range(4):
            self.value_estimator.train(self.state_list, self.reward_list + self.gamma * Vs_)
            
        self._reset()
        
    def _caculate_reward(self):
        # caculate discounted reward with causality (reward to go)
        reward_list = np.array(self.reward_list)
        d_reward = np.zeros_like(reward_list)
        for i in range(len(d_reward)):
            d_reward[i] = np.sum([reward_list[j] * self.gamma**j for j in range(len(reward_list[i:]))])
        return d_reward
    
    def gae(self, Vs, Vs_):
        delta = self.reward_list + self.gamma * Vs_ - Vs # delta_t = r_t + gamma * V_{t+1} - V_t
        A_gae = np.zeros_like(self.reward_list)
        for i in range(len(self.reward_list)):
            A_gae[i] = np.sum([(self.gamma * self.lamda)**j * delta[j] for j in range(len(self.reward_list[i:]))])
        return A_gae

In [3]:
class ValueEstimator:
    def __init__(self, sess, input_shape, lr=1e-4):
        self.sess = sess
        
        self.batch_states = tf.placeholder(tf.float32, [None] + list(input_shape))
        self.V_target = tf.placeholder(tf.float32, [None])
        
        with tf.variable_scope("ValueEstimator"):
            fc1 = fc1 = tf.layers.dense(self.batch_states, 128, activation=tf.nn.relu)
            fc2 = tf.layers.dense(fc1, 256, activation=tf.nn.relu)
            fc3 = tf.layers.dense(fc2, 256, activation=tf.nn.relu)
            self.V_estimate = tf.squeeze(tf.layers.dense(fc3, 1))
            
            mse_loss = tf.losses.mean_squared_error(self.V_target, self.V_estimate)
            l2_loss = tf.add_n([tf.nn.l2_loss(v) for v in tf.trainable_variables() if 'ValueEstimator' in v.name]) * 3e-6
            self.loss = mse_loss + l2_loss
            optimizer = tf.train.AdamOptimizer(learning_rate=lr)
            self.train_step = optimizer.minimize(self.loss)
            
    def predict(self, states):
        return sess.run(self.V_estimate, feed_dict={self.batch_states:states})
    
    def train(self, states, targets):
        loss, _ = sess.run([self.loss, self.train_step], feed_dict={self.batch_states:states, self.V_target:targets})

In [6]:
tf.reset_default_graph()

if not os.path.isdir("models/ppo"):
    os.makedirs("models/ppo")

with tf.device("/gpu:0"):
    
    config = tf.ConfigProto(allow_soft_placement = True)
    with tf.Session(config=config) as sess:
        
        env = gym.make("CartPole-v1")
        state = env.reset()
        action_num = env.action_space.n
        input_shape = env.observation_space.shape
        agent = PPO_Agent(sess, input_shape, action_num)
        
        episodic_reward = tf.get_variable("episodic_reward", (), trainable=False)
        episodic_step = tf.get_variable("episodic_step", (), trainable=False)
        tf.summary.scalar("episode_reward",episodic_reward)
        tf.summary.scalar("episode_step",episodic_step)
        merged = tf.summary.merge_all()
        writer = tf.summary.FileWriter("logs/ppo", sess.graph)
        saver = tf.train.Saver(max_to_keep=10)
        
        sess.run(tf.global_variables_initializer())
        
        e, episode_reward, global_step, episode_step = 0, 0, 0, 0
        
        while True:
            
            action = agent.act(state)
            next_state, reward, done, info = env.step(action)
            # normalize reward
            reward /= 100
            agent.observe(state, action, reward)
            episode_reward += reward
            state = next_state
            episode_step += 1
            global_step += 1
            
            if done:
                if global_step > 100:
                    agent.train()
                state = env.reset()
                summary = sess.run(merged, feed_dict={episodic_reward:episode_reward, episodic_step:episode_step})
                writer.add_summary(summary, global_step=e)
                if e % 10 == 0:
                    writer.flush()
                episode_reward = 0
                episode_step = 0
                e += 1
            
                if e % 200 == 0:
                    saver.save(sess, "models/ppo/model", global_step=e)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


KeyboardInterrupt: 

In [ ]:
tf.reset_default_graph()

with tf.device("/gpu:0"):
    
    config = tf.ConfigProto(allow_soft_placement = True)
    with tf.Session(config=config) as sess:
        
        ckpt = tf.train.get_checkpoint_state(os.path.dirname('./models/ppo/checkpoint'))
        env = gym.make("CartPole-v1")
        state = env.reset()
        action_num = env.action_space.n
        input_shape = env.observation_space.shape
        agent = PPO_Agent(sess, input_shape, action_num, lr=0.001)
        saver = tf.train.Saver()
        saver.restore(sess, ckpt.model_checkpoint_path)
        done = False
        
        for i in range(10):
            l = []
            while not done:
                env.render()
                time.sleep(0.01)
                action = agent.act(state, False)
                state, reward, done, info = env.step(action)
                l.append(reward)
                #print(reward)
                if done:
                    state = env.reset()
                    done = False
                    print(len(l))
                    break
        env.close()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
INFO:tensorflow:Restoring parameters from ./models/ppo\model-1600
500
500
500
500
500
500
500
500
